In [5]:
import os
import json
from docx import Document
from tkinter import *
from tkinter import ttk, messagebox

class SequentialFormFiller:
    def __init__(self, root, config_file):
        self.root = root
        self.root.title("Preenchimento de Documentos")
        self.root.geometry("600x400")
        
        # Carrega configuração
        self.config = self.load_config(config_file)
        self.campos = list(self.config["campos"].items())
        self.current_field_index = 0
        self.responses = {}
        
        # Interface
        self.setup_ui()
        self.show_current_field()
    
    def load_config(self, config_file):
        """Carrega o arquivo de configuração JSON"""
        with open(config_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def setup_ui(self):
        """Configura a interface do usuário"""
        self.main_frame = ttk.Frame(self.root, padding="20")
        self.main_frame.pack(fill=BOTH, expand=True)
        
        # Área do campo atual
        self.field_frame = ttk.Frame(self.main_frame)
        self.field_frame.pack(fill=BOTH, expand=True, pady=20)
        
        self.field_label = ttk.Label(self.field_frame, font=('Arial', 12))
        self.field_label.pack(pady=10)
        
        self.entry = ttk.Entry(self.field_frame, font=('Arial', 12))
        self.entry.pack(fill=X, pady=5)
        
        # Botões de navegação
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(fill=X, pady=10)
        
        self.prev_btn = ttk.Button(
            self.button_frame, 
            text="Anterior", 
            command=self.prev_field,
            state=DISABLED
        )
        self.prev_btn.pack(side=LEFT, padx=5)
        
        self.next_btn = ttk.Button(
            self.button_frame, 
            text="Próximo", 
            command=self.next_field
        )
        self.next_btn.pack(side=LEFT, padx=5)
        
        # Botão de finalização (inicialmente oculto)
        self.finish_btn = ttk.Button(
            self.main_frame, 
            text="Gerar Documento", 
            command=self.generate_document,
            state=DISABLED
        )
    
    def show_current_field(self):
        """Mostra o campo atual na interface"""
        if self.current_field_index >= len(self.campos):
            return
            
        # Limpa o frame
        for widget in self.field_frame.winfo_children():
            widget.destroy()
        
        field_name, field_config = self.campos[self.current_field_index]
        
        # Atualiza a interface
        self.field_label = ttk.Label(
            self.field_frame, 
            text=field_config["rotulo"],
            font=('Arial', 12)
        )
        self.field_label.pack(pady=10)
        
        self.entry = ttk.Entry(
            self.field_frame, 
            font=('Arial', 12)
        )
        self.entry.pack(fill=X, pady=5)
        
        # Preenche com valor já respondido se existir
        if field_name in self.responses:
            self.entry.insert(0, self.responses[field_name])
        
        # Atualiza estado dos botões
        self.prev_btn.config(state=NORMAL if self.current_field_index > 0 else DISABLED)
        
        if self.current_field_index == len(self.campos) - 1:
            self.next_btn.pack_forget()
            self.finish_btn.pack(pady=20)
            self.finish_btn.config(state=NORMAL)
        else:
            self.finish_btn.pack_forget()
            self.next_btn.pack(side=LEFT, padx=5)
    
    def save_current_response(self):
        """Salva a resposta do campo atual"""
        field_name = self.campos[self.current_field_index][0]
        self.responses[field_name] = self.entry.get()
    
    def next_field(self):
        """Vai para o próximo campo"""
        self.save_current_response()
        self.current_field_index += 1
        self.show_current_field()
    
    def prev_field(self):
        """Volta para o campo anterior"""
        self.save_current_response()
        self.current_field_index -= 1
        self.show_current_field()
    
    def generate_document(self):
        """Gera o documento preenchido"""
        try:
            self.save_current_response()
            
            # Verifica campos obrigatórios
            for field_name, field_config in self.campos:
                if field_config["obrigatorio"] and not self.responses.get(field_name):
                    messagebox.showerror(
                        "Campo obrigatório", 
                        f"O campo '{field_config['rotulo']}' não foi preenchido!")
                    return
            
            # Carrega o template
            template_path = os.path.join("templates", os.path.basename(self.config_file).replace(".json", ".docx"))
            doc = Document(template_path)
            
            # Substitui os placeholders
            for paragraph in doc.paragraphs:
                for field_name, response in self.responses.items():
                    if field_name in paragraph.text:
                        paragraph.text = paragraph.text.replace(field_name, response)
            
            # Substitui em tabelas
            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        for field_name, response in self.responses.items():
                            if field_name in cell.text:
                                cell.text = cell.text.replace(field_name, response)
            
            # Salva o documento
            output_dir = "output"
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, os.path.basename(template_path).replace(".docx", "_PREENCHIDO.docx"))
            doc.save(output_path)
            
            messagebox.showinfo(
                "Sucesso!", 
                f"Documento gerado com sucesso em:\n{output_path}")
            
            self.root.destroy()
            
        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro ao gerar o documento:\n{str(e)}")

def main():
    # Aqui você pode selecionar qual JSON usar ou fazer um menu para seleção
    config_file = "config/campos_ATA.json"  # Altere para o seu arquivo JSON
    
    root = Tk()
    app = SequentialFormFiller(root, config_file)
    root.mainloop()

if __name__ == "__main__":
    main()